In [1]:
import os
import pandas as pd
import numpy as np
from pid_sm import PIDController
from concurrent.futures import ProcessPoolExecutor

# Function to simulate the system response using the PID controller
def simulate_system_response(kp, ki, kd, setpoint, object_centers):
    pid_controller = PIDController(kP=kp, kI=ki, kD=kd)

    data = {'Input': [], 'kP': [], 'kI': [], 'kD': [], 'Output': []}

    for object_center in object_centers:
        error = setpoint - object_center
        output = pid_controller.update(error)

        # Append values to the data dictionary
        data['Input'].append(error)
        data['kP'].append(kp)
        data['kI'].append(ki)
        data['kD'].append(kd)
        data['Output'].append(output)

    return pd.DataFrame(data)

# Generate input values
object_centers = np.arange(0, 641, 10)

# Setpoint for all cases
setpoint = 320

# Get user input for kp, ki, and kd values
kp = float(input("Enter the value for kp: "))
ki = float(input("Enter the value for ki: "))
kd = float(input("Enter the value for kd: "))

# Specify the folder path
folder_path = '../outPutData/'  # Adjust the relative path as needed

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Create a string to append to the file name
param_string = f"_kp_{kp}_ki_{ki}_kd_{kd}"

# Using ProcessPoolExecutor for parallel execution
with ProcessPoolExecutor() as executor:
    # Simulate system response for user-specified PID parameters
    futures = [executor.submit(simulate_system_response, kp, ki, kd, setpoint, object_centers)]
    df_list = [future.result() for future in futures]

# Concatenate DataFrames from different processes
df = pd.concat(df_list, ignore_index=True)

# Print the head of the DataFrame
print("\nHead of the DataFrame:")
print(df.head())

# Save the DataFrame to an Excel file in the specified folder
excel_filename = os.path.join(folder_path, f'system_response_data{param_string}.xlsx')
df.to_excel(excel_filename, index=False)

print(f'Data saved to {excel_filename}')


ValueError: could not convert string to float: ''